In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
%matplotlib inline
# import warnings
# warnings.filterwarnings('ignore')
import optuna

from sklearn.model_selection import RepeatedStratifiedKFold, GroupKFold, KFold
# from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, 
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import catboost as cb

# optuna.logging.set_verbosity(optuna.logging.WARNING)
import warnings
# warnings.filterwarnings('ignore')

# from xgboost import *
pd.set_option("display.max_columns", None)

from View import MyClass

In [2]:
mc = MyClass()
print(mc.__doc__)

Визуализация и предобработка данных
# Первичный обзор работа с памятью
* Метод "view_csv" принимает DataFrame и выводит данные о нем, переводит название столбцов в lower
возвращает data.columns, categorical, numerical
* Метод optim_memory оптимизирует память занимаемую данными, принимает DataFrame, возвращает DataFrame

# Статистика
* Метод "stat_frame" принимает DataFrame и возвращает DataFrame со статистиками
* Метод "corr" принимает DataFrame и target : str
* Метод corr_sign принимает data: DataFrame, drop_sign:List ( корреляция между признаками)
* Метод nonlianer_comun принимает data: DataFrame, numerical: list(список числовых признаков
target: str(по умолчанию "class"), num_sign: int(кол-во столбцов с нелинейной связью)
* Метод "percentile_99_1" принимает data: DataFrame, data_stat : DataFrame( возвращает метод "stat_frame"), flag: bool(
если флаг равен True возвращает фрейм где максимум заменен на 99 перцентиль, а минимум на 1 перцентиль)
по умолчанию flag = False
возвращат данны

In [5]:
os.listdir('.')

['.ipynb_checkpoints',
 'analisys.ipynb',
 'baseline.ipynb',
 'catboost_info',
 'data',
 'ensemble_regression.ipynb',
 'optiver-trading-at-the-close-publicleaderboard-2023-10-30T09_57_54.csv',
 'optiver-trading-at-the-close-publicleaderboard-2023-10-30T09_57_54.zip',
 'optiver2023',
 'public_timeseries_testing_util.py',
 'sub_base.csv',
 'sub_base1.csv',
 'train_new.csv']

In [6]:
train = pd.read_csv('data/train_new.csv')
test = pd.read_csv('data/test.csv')
sub = pd.read_csv('data/sample_submission.csv')
rev_target = pd.read_csv('data/revealed_targets.csv')
df = pd.read_csv("optiver-trading-at-the-close-publicleaderboard-2023-10-30T09_57_54.csv")

In [6]:
np.__version__

'1.21.5'

In [5]:
sub.head()

,time_id,row_id,target
0,26290,478_0_0,1
1,26290,478_0_1,1
2,26290,478_0_2,1
3,26290,478_0_3,1
4,26290,478_0_4,1


In [ ]:
train.columns, categorical, numerical = mc.view_csv(train)

# удаление одинаковых строк

In [ ]:
# train = train.dropna(subset=['target'])

In [ ]:
# # одинаковые строки
# to_check = pd.merge(train.drop(columns = ['target'], axis = 1), test, how='inner', on='row_id')
# to_check.shape

In [ ]:
# train = train[~train['row_id'].isin(test['row_id'])]

In [ ]:
# to_check = pd.merge(train.drop(columns = ['target'], axis = 1), test, how='inner', on='row_id')
# to_check.shape

In [ ]:
# train = train.drop_duplicates()

In [ ]:
# train.shape

In [ ]:
# train.to_csv('train_new.csv', index=False)

# &&&&&&&&&&&&&&&&&&&&&&&&&&&

In [ ]:
mc.corr(train, 'target')

In [ ]:
mc.corr_sign(train, drop_sign=[])

* много признаков коррелирующих между собой

In [ ]:
#     def nonlianer_comun(data, numerical, target="class", num_sign=20) -> list:
#         parameters = {"max_depth": 6, "n_estimators": 25, "random_state": 27, "n_jobs": 2}
#         data = data.copy()
#         data = data[numerical]
#         for i in data.columns:
#             data[i] = data[i].fillna(data[i].median())
#         if data[target].nunique() > 10:
#             forest = RandomForestRegressor(**parameters)
#         else:
#             forest = RandomForestClassifier(**parameters)
#         forest.fit(data.drop(target, axis=1), data[target])

#         # numerical original
#         top = []
#         n_top = num_sign
#         importances = forest.feature_importances_
#         idx = np.argsort(importances)[::-1][0:n_top]
#         feature_names = data.drop(target, axis=1).columns

#         plt.figure(figsize=(20, 5))
#         sns.barplot(x=feature_names[idx], y=importances[idx], palette="viridis")
#         top.append(feature_names[idx])
#         plt.title(f"What {num_sign} are the top important features to start with?", size=14)
#         plt.xticks(rotation=90)
#         d = {k: v for k, v in list(zip(list(top[0]), list(importances)))}
#         print(d)
#         return top

In [ ]:
# nonlianer_comun(train, numerical, 'target')

In [ ]:
data_stat = mc.stat_frame(train)

In [ ]:
data_stat.head(train.shape[1])

In [ ]:
train['target'].nunique()

In [ ]:
# замена на 99 перцентиля на макс, 1 min
data_perc = train[numerical].copy()
for col in tqdm(data_perc.columns):
    len_more_quant = data_perc.loc[data_perc[col] > data_perc[col].quantile(0.99)].shape[0]
    len_less_quant = data_perc.loc[data_perc[col] < data_perc[col].quantile(0.01)].shape[0]
    if len_more_quant > 0:
        data_perc.loc[data_perc[col] > data_perc[col].quantile(0.99), col] = data_perc[col].quantile(0.99)
    if len_less_quant > 0:
        data_perc.loc[data_perc[col] < data_perc[col].quantile(0.01), col] = data_perc[col].quantile(0.01)

In [ ]:
# train = train.dropna()

In [ ]:
# train['target'].nunique()

In [ ]:
train.shape

# memory

In [ ]:
# train = mc.optim_memory(train)

In [ ]:
# train.to_csv("train_new.csv", index=False)

# kmeans

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

In [ ]:
train = mc.median_mode(train)

In [ ]:
def find_num_cluster(data):
    distortions = []
    inertias = []
    mapping1 = {}
    mapping2 = {}
    K = range(1, 10)

    for k in K:
        print(f"fit model, cluster: {k}")
        # Building and fitting the model
        kmeanModel = KMeans(n_clusters=k).fit(data)
    #     kmeanModel.fit(X)
        
        distortions.append(sum(np.min(cdist(train, kmeanModel.cluster_centers_,
                                            'cosine'), axis=1)) / train.shape[0])
        inertias.append(kmeanModel.inertia_)

        mapping1[k] = sum(np.min(cdist(train, kmeanModel.cluster_centers_,
                                       'cosine'), axis=1)) / train.shape[0]
        mapping2[k] = kmeanModel.inertia_
        
        
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('Values of K')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method using Distortion')
    plt.show()

In [ ]:
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

# графики

In [ ]:
np.zeros(5) * [1,2,3,4,5]

In [ ]:
sns.scatterplot(data=train['target'].unique())

In [ ]:
sns.scatterplot(data=train['target'].sample(10000).unique())

In [ ]:
np.ones(len(train['target'].unique()))* train['target'].std()

In [ ]:
train.columns

In [ ]:
cols = ['seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id']

In [ ]:
for i, col in enumerate(train[cols]):
    plt.figure(i)

    sns.scatterplot(data=train.sample(10000), x='target', y=col, sizes=(20, 200))

In [ ]:
for i, col in enumerate(train[cols]):
    plt.figure(i)
    sns.countplot(x=col, data=train[cols].sample(10000))